## **DATA LOADER**

In [2]:
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tqdm import tqdm
import matplotlib.pyplot as plt

In [4]:
def load_data(folder_path, csv_path, is_train=True, max_frames=30):
    df = pd.read_csv(csv_path)
    video_ids = df['video_id'].values
    labels = df['label'].values if is_train else None

    data = []
    label_list = []

    # Add tqdm progress bar
    for i, video_id in tqdm(enumerate(video_ids), total=len(video_ids), desc="Loading images"):
        video_folder = os.path.join(folder_path, str(video_id))
        if not os.path.exists(video_folder):
            print(f"Folder {video_folder}, No such file or directory")
            continue

        frames = []
        # Add nested progress bar for image loading if you want to see per-folder progress
        # for img_name in tqdm(sorted(os.listdir(video_folder)), desc=f"Loading folder {video_id}", leave=False):
        for img_name in sorted(os.listdir(video_folder)):
            img_path = os.path.join(video_folder, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (64, 64))  
                frames.append(img)

        if len(frames) > max_frames:
            frames = frames[:max_frames]

        if len(frames) > 0:
            data.append(np.array(frames))  
            if is_train:
                label_list.append(labels[i])

    if is_train:
        return np.array(data, dtype=object), np.array(label_list)
    else:
        return np.array(data, dtype=object), None

In [ ]:

train_folder = "E:/Academic/CS4203 - Research and Development Project/DataSet/Jester/20BN-JESTER/Train"
test_folder = "E:/Academic/CS4203 - Research and Development Project/DataSet/Jester/20BN-JESTER/Test"
train_csv = "E:/Academic/CS4203 - Research and Development Project/DataSet/Jester/20BN-JESTER/Train.csv"
test_csv = "E:/Academic/CS4203 - Research and Development Project/DataSet/Jester/20BN-JESTER/Test.csv"

X_train, y_train = load_data(train_folder, train_csv, is_train=True)

unique_labels = np.unique(y_train)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
y_train_encoded = np.array([label_mapping[label] for label in y_train])
y_train_onehot = to_categorical(y_train_encoded, num_classes=len(unique_labels))


X_test, _ = load_data(test_folder, test_csv, is_train=False)

In [18]:

X_train = np.array([np.array(frames) / 255.0 for frames in X_train], dtype=object)
X_test = np.array([np.array(frames) / 255.0 for frames in X_test], dtype=object)

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_timesteps = 30
X_train = pad_sequences(X_train, maxlen=max_timesteps, padding='post', dtype='float32')
X_test = pad_sequences(X_test, maxlen=max_timesteps, padding='post', dtype='float32')

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 64, 64, 3))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 64, 64, 3))

## **CNN-LSTM**

In [ ]:
model = models.Sequential([
    # CNN Layers
    layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu'), input_shape=(None, 64, 64, 3)),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu')),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.GlobalAveragePooling2D()),  # Reduces parameters compared to Flatten

    # LSTM Layers
    layers.LSTM(64, return_sequences=True, activation='tanh'),
    layers.LSTM(32, activation='tanh'),

    # Dense Layers
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(unique_labels), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

In [ ]:
# Training
history = model.fit(X_train, y_train_onehot, epochs=50, batch_size=8, validation_split=0.2)

In [ ]:
predictions = model.predict(X_test)
predicted_labels = [unique_labels[np.argmax(pred)] for pred in predictions]

test_df = pd.read_csv(test_csv)
test_df['label'] = predicted_labels
test_df.head()

In [ ]:
import matplotlib.pyplot as plt

# Plot akurasi
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training Accuracy')
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Loss')
plt.show()

In [ ]:
final_accuracy = history.history['accuracy'][-1]
print(f"Final Training Accuracy: {final_accuracy * 100:.2f}%")

for epoch, acc in enumerate(history.history['accuracy'], start=1):
    print(f"Epoch {epoch}: Training Accuracy = {acc * 100:.2f}%")

final_loss = history.history['loss'][-1]
print(f"Final Training Loss: {final_loss:.4f}")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

# Plot loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

## **CNN-GRU**

In [ ]:
model = models.Sequential([
    # CNN Layers
    layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu'), input_shape=(None, 64, 64, 3)),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu')),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.GlobalAveragePooling2D()),  # Reduces parameters compared to Flatten

    # GRU Layers
    layers.GRU(64, return_sequences=True, activation='tanh'),
    layers.GRU(32, activation='tanh'),

    # Dense Layers
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(unique_labels), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

In [ ]:
# Training the model
history = model.fit(X_train, y_train_onehot, epochs=50, batch_size=4, validation_split=0.2)

In [ ]:
# Predictions on test data
predictions = model.predict(X_test)
predicted_labels = [unique_labels[np.argmax(pred)] for pred in predictions]

test_df = pd.read_csv(test_csv)
test_df['label'] = predicted_labels
test_df.head()

In [ ]:
# Plot Training and Validation Accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
# Plot Training and Validation Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Print final metrics
final_accuracy = history.history['accuracy'][-1]
final_loss = history.history['loss'][-1]
print(f"Final Training Accuracy: {final_accuracy * 100:.2f}%")
print(f"Final Training Loss: {final_loss:.4f}")